# Biased Constraint Demotion: a Python Program

## Introduction

In the Optimality Theory of linguistics, there exists a Universal Grammar of constraints common to all natural languages, and the grammar differentiating different languages is merely the or ordering of these constraints. Using constraints instead of rules means that the output that ends up being the "winner," or grammatical, among competing candidates *may* violate some constraints, so long as it's still preferable to the other candidates. Therefore, a learner's task is to find the correct order of the constraints for the target language (Danis, 2020b). 

Bruce Tesar and Paul Smolensky developed the Recursive Constraint Demotion (RCD) learning algorithm that accomplishes this from positive and negative input-output data, which finds a stratified hierarchy consistent to the data (Tesar and Smolensky, 2004). However, since scientists, computer learners and human learners cannot feasibly have access to the entire corpus of possible data, often there are multiple hierarchies that work, which can be potentially eliminated as more data comes in. If a learned grammar is broader than the actual grammar, then no positive evidence can contradict that hypothesis. Therefore narrower grammars should be preferred during learning (Prince and Tesar, 2004). 

Alan Prince and Bruce Tesar developed the Biased Constraint Demotion (BCD) algorithm to attempt this, utilizing the difference between markedness and faithfulness constraints (Prince and Tesar, 2004). Markedness constraints look only at characteristics of the output, while faithfulness constraints look at input-output disparities.

The Python class demonstrated here aims to execute their outlined algorithm.

## Demonstration

The functionality is held within a class called BCD.

### Importing

To use BCD from the Python shell while in this directory, use ```from src.BCD import BCD```

Make sure you have the *src* directory in the same directory as this notebook. Then run the following cell:

In [1]:
from src.BCD import BCD

BCD successfully imported!


### Lardil Example

The following demonstrates the program using the Lardil example used in *RCD the Movie,* a resource developed by Alan Prince (2009). As he uses RCD, the resulting heirarchy will be different than what was concluded there, but it will result in a higher R-measure, which is preferred by BCD.

To perform BCD, we need mark-data pairs (mdps), a collection of data containing an optimal output, a losing candidate, and for ever constraint in consideration, whether the winner or loser, if either at all, is preferred. There can be multiple pairs with the same optimal output and different losing candidates. This program reads this data in the form of a constraint table from a CSV file using Pandas. The required format is a header, first listing the input (for bookkeeping purposes), the winner, and the loser in that order, followed by constraint names. Each row thereafter represents an mdp, where the value for each constraint is 'W' if the winner is preferred by the constraint, 'L' if the loser is preferred, and any other value is taken as no preference.

Each constraint name must begin with 'm:' or 'f:', denoting it as either a markedness or faithfulness constraint, which needs to be known for the algorithm to work. Other markers are allowed as well, but anything not beginning with 'f:' is assumed to be a markedness constraint. *note: therefore if you forget to add markers at all, everything will be assumed a markedness constraint **and this will run like RCD**?*

In [2]:
lardil = BCD()
lardil.loadCt('data/ldl2.csv')
display(lardil.ct)
print('Markedness Constraints:', lardil.markednessConstraints)

# you can also load the constraint table upon initialization with the following shortcut:
# lardil = BCD(ctPath='data\ldl2.csv')
# display(lardil.ct)
# print('Markedness Constraints:', lardil.markednessConstraints)

,Input,Winner,Loser,f:DepV,m:*Cmplx,af:Free-V,p:Algn,m:CdCnd,f:DepC,f:Max,p:LxPr,m:NoCoda
0,kentapal,.ken.ta.pal.,.ke.nA.ta.pa.<l>,W,NaN,NaN,W,NaN,NaN,W,NaN,L
1,waNalk,.wa.Nal.<k>,.wa.Nalk.,NaN,W,NaN,L,W,NaN,L,NaN,NaN
2,waNalk,.wa.Nal.<k>,.wa.Nal.kA.,W,NaN,NaN,NaN,NaN,NaN,L,NaN,NaN
3,Naluk,.Na.lu.<k>,.Na.luk.,NaN,NaN,NaN,L,W,NaN,L,NaN,W
4,maR,.maR.TA.,.maR.,L,NaN,NaN,NaN,NaN,L,NaN,W,NaN
5,maR,.maR.TA.,.ma.RA.,NaN,NaN,NaN,W,NaN,L,NaN,NaN,L
6,Relk,.Rel.kA.,.Rel.<k>,L,NaN,NaN,NaN,NaN,NaN,W,W,NaN
7,yak,.ya.kA.,.yak.CA.,NaN,NaN,NaN,L,W,W,NaN,NaN,W
8,kaN,.kaN.KA.,.ka.NA.,NaN,NaN,NaN,W,NaN,L,NaN,NaN,L
9,yiliyili,.yi.li.yil.<i>,.yi.li.yi.li.,NaN,NaN,W,L,NaN,NaN,L,NaN,L


Markedness Constraints: {'m:CdCnd', 'm:NoCoda', 'p:LxPr', 'af:Free-V', 'p:Algn', 'm:*Cmplx'}


The doBCD() method then finds a stratified hierarchy. Biased Constraint Demotion places f-constraints as futherest down as possible, even when both m- and f- constraints *can* be placed at a higher stratum together. To assess how well this is done, Prince and Tesar define a new metric, R-measure (2004):

 * **R-measure**. The r-measure for a constraint hierarchy is determined by adding, for each faithfulness constraint in the hierarchy, the number of markedness constraints that dominate that faithfulness constraint.


Higher R-measures are preferred. They also list four principles for modifying RCD into BCD (Prince and Tesar, 2004):

1. **Faithfulness Delay**. Only place faithfulness constraints if no markedness constraints are available to be placed in the hierarchy.
    * (This also means than markedness and faithfulness constraints will never be in the same stratum.)
2. **Avoid the inactive**. When placing faithfulness constraints into the hierarchy, if possible only place those that prefer some winner. If the only available faithfulness constraints prefer no remaining winners, then place all of them into the hierarchy.
3. **Smallest efffective F sets**. When placing faithfulness constraints into the hierarchy, place the smallest set of F constraints that frees up some markedness constraint.
4. **Richest Markedness Cascade**. When placing faithfulness constraints into the hierarchy, if more than one F-set freeing up some markedness constraint is of smallest size, then place the F-set that yields the largest set of M constraints in contiguous subsequent strata, i.e. until another F constraint must be ranked. If there is more than one of these, pick one at random.

The result is stored in the strata variable; or you can print a formatted version with the printStrata() function. The original constraint tableau is preserved, but you can view and save an organized version with organizeTableau() and saveOrganizedTableau() respectively. The constraints (columns) are reordered by stratum, the mdps (rows) are reordered with those resolved by the highest strata placed earlier, and a new *Rank* column is added to indicate which stratum resolved the pair.

In [3]:
print('lardil.strata before running doBCD():', lardil.strata)
print()

lardil.doBCD()
lardil.printStrata()
display(lardil.organizeTableau(lardil.ct, lardil.strata))
lardil.saveOrganizedTableau('results/ldl2_organized.csv')

print('R-measure =', lardil.calculateRMeasure())

lardil.strata before running doBCD(): []

STRATA:
Rank 1 {'m:*Cmplx', 'm:CdCnd', 'p:LxPr'}
Rank 2 {'f:DepV'}
Rank 3 {'af:Free-V'}
Rank 4 {'p:Algn'}
Rank 5 {'m:NoCoda'}
Rank 6 {'f:Max', 'f:DepC'}


,Input,Winner,Loser,m:*Cmplx,m:CdCnd,p:LxPr,f:DepV,af:Free-V,p:Algn,m:NoCoda,f:Max,f:DepC,Rank
1,waNalk,.wa.Nal.<k>,.wa.Nalk.,W,W,,,,L,,L,,1
14,wiTe,.wi.Te.,.wiT.<e>,,,W,,L,W,W,W,,1
3,Naluk,.Na.lu.<k>,.Na.luk.,,W,,,,L,W,L,,1
4,maR,.maR.TA.,.maR.,,,W,L,,,,,L,1
13,muNkumuNku,.muN.ku.mu.<N><k><u>,.muN.ku.muN.<k><u>,,W,,,,,W,L,,1
6,Relk,.Rel.kA.,.Rel.<k>,,,W,L,,,,W,,1
7,yak,.ya.kA.,.yak.CA.,,W,,,,L,W,,W,1
10,yukarpa,.yu.kar.<pa>,.yu.karp.<a>,W,W,,,,,,L,,1
0,kentapal,.ken.ta.pal.,.ke.nA.ta.pa.<l>,,,,W,,W,L,W,,2
11,yukarpa,.yu.kar.<pa>,.yu.kar.pA.<a>,,,,W,,,,L,,2


R-measure = 15


The resulting ranking is slightly different than what RCD would give, notably f:Max and f:DepC are placed last instead, but this ranking is still consistent with the data (Danis, 2020a). What the rankings mean essentially is that based on the given data, m:CdCnd, m:\*Cmplx, and p:LxP can be placed in any order between the three of them, but they all must dominate every other constraint. After those three are placed, then f:DepV is the next dominant, then af:Free-V, and so on. f:Max and f:DepC can come in any order at the end. 

The R-measure for these strata is 15, which beats out RCD's 12. 

### Starting from a violation tableau

The program works also by feeding in a violation tableau instead of a constraint tableau (also in CSV form). In the violation tableau, after the header, each row starts with the input, the output, whether or not it is the optimal output (any mark in this column is considered a yes). It is expected that there be multiple rows per input with different outputs, and that exactly one of these is optimal. Columns following those three are for each constraint in quesiton, with an interger value representing how many violations of said constraint is made by this input-output pair. The header consists of 'Input,' 'Output,' 'Optimal,' and then each constraint name marked with 'm:' or 'f:'.

The following is a toy example from lecture, simply to demonstrate that generating a consistent ranking from a violation tableau is feasible (Danis, 2020b):

In [4]:
anpa = BCD()
anpa.loadVt('data/vt_from_intro_to_OT.csv')
display(anpa.vt)
anpa.generateCtFromVt()
print('Constraint tableau:')
display(anpa.ct)
# you can also save this constraint tableau with saveCt()
print('Markedness Constraints:', anpa.markednessConstraints)

# you can also load the constraint table upon initialization with the following shortcut:
# anpa = BCD(vtPath='data/vt_from_intro_to_OT.csv')
# display(anpa.vt)
# anpa.generateCtFromVT()
# display(anpa.ct)
# print('Markedness Constraints:', anpa.markednessConstraints)

,Input,Output,Optimal,f:Don'tDelete,f:Don'tInsert,f:BeIdentical,m:AgreePlace,m:Onset
0,/anpa/,anpa,NaN,0,0,0,1,1
1,/anpa/,ampa,NaN,0,0,1,0,1
2,/anpa/,anapa,NaN,0,1,0,0,1
3,/anpa/,ana,NaN,1,0,0,0,1
4,/anpa/,apa,NaN,1,0,0,0,1
5,/anpa/,npa,NaN,1,0,0,1,0
6,/anpa/,mpa,NaN,1,0,1,0,0
7,/anpa/,napa,NaN,1,1,0,0,0
8,/anpa/,na,NaN,2,0,0,0,0
9,/anpa/,pa,NaN,2,0,0,0,0


Constraint tableau:


,Input,Winner,Loser,f:Don'tDelete,f:Don'tInsert,f:BeIdentical,m:AgreePlace,m:Onset
0,/anpa/,tampa,anpa,,L,L,W,W
1,/anpa/,tampa,ampa,,L,,,W
2,/anpa/,tampa,anapa,,,L,,W
3,/anpa/,tampa,ana,W,L,L,,W
4,/anpa/,tampa,apa,W,L,L,,W
5,/anpa/,tampa,npa,W,L,L,W,
6,/anpa/,tampa,mpa,W,L,,,
7,/anpa/,tampa,napa,W,,L,,
8,/anpa/,tampa,na,W,L,L,,
9,/anpa/,tampa,pa,W,L,L,,


Markedness Constraints: {'m:AgreePlace', 'm:Onset'}


In [5]:
anpa.doBCD()
anpa.printStrata()
display(anpa.organizeTableau(anpa.ct, anpa.strata))

print('R-measure =', anpa.calculateRMeasure())

STRATA:
Rank 1 {'m:AgreePlace', 'm:Onset'}
Rank 2 {"f:Don'tDelete"}
Rank 3 {"f:Don'tInsert"}
Rank 4 {'f:BeIdentical'}


,Input,Winner,Loser,m:AgreePlace,m:Onset,f:Don'tDelete,f:Don'tInsert,f:BeIdentical,Rank
0,/anpa/,tampa,anpa,W,W,,L,L,1
1,/anpa/,tampa,ampa,,W,,L,,1
2,/anpa/,tampa,anapa,,W,,,L,1
3,/anpa/,tampa,ana,,W,W,L,L,1
4,/anpa/,tampa,apa,,W,W,L,L,1
5,/anpa/,tampa,npa,W,,W,L,L,1
10,/anpa/,tampa,tanpa,W,,,,L,1
6,/anpa/,tampa,mpa,,,W,L,,2
7,/anpa/,tampa,napa,,,W,,L,2
8,/anpa/,tampa,na,,,W,L,L,2


R-measure = 6


### findPotentialMinFaithSubsets()

The following dataset comes from Appendix 2 of Prince and Tesar (2004). Their contrived example is designed to show the greedy approach used here produces a smaller, less preferable R-measure, but Prince and Tesar themselves state "Pursuing this potential trade-off with full vigor is computationally unattractive, and therefore offensive to mental realism. Until there is evidence that it is meaningful or even likely to occur with realistic F and M constraints, we prefer to develop a version of BCD that simply favors smaller F-gangs."

Here, the example is being used simply to show that findMinFaithSubset() does what it claims, as none of the previous examples use the function. While the strata achieved here is different from the example strata they give, one will find that if one manually performs BCD, this stratified hierarchy (or an equivalent) will be produced.

In [6]:
appendix2 = BCD(ctPath='data\prince_and_tesar_appendix_2.csv')
display(appendix2.ct)
print('Markedness Constraints:', appendix2.markednessConstraints)
print()

appendix2.doBCD()
appendix2.printStrata()
display(appendix2.organizeTableau(appendix2.ct, appendix2.strata))

print('R-measure =', appendix2.calculateRMeasure())

,Input,Winner,Loser,m:M1,m:M2,m:M3,m:M4,f:F1,f:F2,f:F3,f:F4
0,I1,W1,L1,L,NaN,NaN,NaN,W,W,NaN,NaN
1,I2,W2,L2,L,NaN,NaN,NaN,W,NaN,W,NaN
2,I3,W3,L3,L,NaN,NaN,NaN,W,NaN,NaN,W
3,I4,W4,L4,NaN,L,L,L,NaN,W,W,NaN
4,I5,W5,L5,NaN,L,L,L,NaN,W,NaN,W
5,I6,W6,L6,NaN,L,L,L,NaN,NaN,W,W


Markedness Constraints: {'m:M1', 'm:M2', 'm:M3', 'm:M4'}

STRATA:
Rank 1 {'f:F1'}
Rank 2 {'m:M1'}
Rank 3 {'f:F4', 'f:F3'}
Rank 4 {'m:M3', 'm:M2', 'm:M4'}
Rank 5 {'f:F2'}


,Input,Winner,Loser,f:F1,m:M1,f:F4,f:F3,m:M3,m:M2,m:M4,f:F2,Rank
0,I1,W1,L1,W,L,,,,,,W,1
1,I2,W2,L2,W,L,,W,,,,,1
2,I3,W3,L3,W,L,W,,,,,,1
3,I4,W4,L4,,,,W,L,L,L,W,3
4,I5,W5,L5,,,W,,L,L,L,W,3
5,I6,W6,L6,,,W,W,L,L,L,,3


R-measure = 6


## Implementation deviations from Prince and Tesar

Prince and Tesar's version of the BCD Algorithm is copied below (2004):

**BCD( ):**

    Repeat (Until all constraints have been ranked)  
        Set NoL to be the constraints not yet ranked that prefer no losers  
        If at least one of NoL is a markedness constraint  
            Set RankNext to be the set of markedness constraints in NoL  
        Else
            If at least one of NoL prefers a winner
                Set RankNext to be the constraint set returned by Find-minimal-faith-subset(NoL)
            Else
                Set RankNext to be all the constraints of NoL
            End-if
        End-if
        Place RankNext in the next stratum
        Delete all mark-data pairs where the winner is preferred by a constraint in RankNext
    End-Repeat

**Find_minimal_faith_subset(NoL):**

    Set ActiveFaith to be those members of NoL that prefer a winner in one of the remaining md-pairs
    Set FSetSize to start at 0
    Repeat (until a constraint set freeing a markedness constraint is found)
        Increase FSetSize by one
        Generate all subsets of ActiveFaith that are of size FSetSize
        For each such subset FaithSet-x
            If FaithSet-x frees up a markedness constraint, add it to FreeingFaithSets
        End-for
    End-Repeat
    If FreeingFaithSets contains only one faithfulness constraint set
        Set BestFaithSet to be the constraint set in FreeingFaithSets
    Else
        Set BestFaithSet to the constraint set returned by Select-best-faith-set(FreeingFaithSets)
    End-if
    Return(BestFaithSet)

**Select-best-faith-set(FreeingFaithSets):**

    For each set FaithSet-y in FreeingFaithSets
        Place FaithSet-y in the next stratum of the constraint hierarchy under construction
        [!!!] Continue BCD forward until another faithfulness constraint must be placed in the hierarchy
        Set Value-y to the number of markedness constraints ranked after the placement of FaithSet-y
    End-for
    Set BestFaithSet to the member FaithSet-y of FreeingFaithSets with the largest Value-y
    If there is a tie for the largest Value-y
        pick one of them at random
    End-if
    Return(BestFaithSet)
    
    
After initial attempts to stay faithful to this structure, the implementation of this particular program deviates a little. Namely, the line in Select-best-faith-set() marked with \[!!!\] (emphasis mine) does not translate easily into code, as it basically asked for recursive execution of BCD on multiple candidates, with slightly different return values than BCD() itself. Therefore, while what is here is by no means optimal nor the neatest code, some things have been modified for better modularity, readability, and reusage.

In this implementation, the main method doBCD() is analogous to Prince and Tesar's BCD(), but the helper methods deviate slightly:

* **doBCD()**: The main function. As long as there are constraints yet to be placed, it iteratively finds the next stratum.
* **findNextStratum()**: This is called from both doBCD() and findMinFaithSubset(). This determines if there is one definitive set to put as the next stratum. If so, it places it and continues. If there are multiple possible sets, then it stores those sets in a stack for findMinFaithSubset() to pick out the best.
* **findPotentialMinFaithSubsets()**: This is called from findNextStratum(). If no markedness constraints can be ranked at a given moment, this finds the smallest possible set(s) of faithfulness constraints that can be placed. It may return a single set or a list of potential sets, which are then processed by findMinFaithSubset().
* **findMinFaithSubset()**: This is called from doBCD(). As long as possible strata are being compared, this performs BCD using those strata non-destructively to find which frees the most markedness constraints. This utilizes findNextStratum() as well. This also preserves down-the-line stratum for the winning set so that those iterations do need to be redone.

There are also helper functions like fusion() as well. For more documentation, consult the code itself.

## Future Work

* runtime analysis and imporvemnts
* more datasets
* better packaging and documentation
* error and exception handling for misusage and usage on inconsistent data

## Bibliography

Danis, N. (2020a, April 23). *BCD* \[Excel Workbook\]. (class notes)

Danis, N. (2020b, April 1). *Intro to OT* \[Excel Workbook\]. (class notes)

Prince, A. (2009). RCD–the movie. ROA-1057.

Prince, A., & Tesar, B. (2004). Learning phonotactic distributions. *Constraints in phonological acquisition*, 245-291.

Tesar, B., & Smolensky, P. (1998). Learnability in optimality theory. *Linguistic Inquiry, 29(2)*, 229-268.